In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

ModuleNotFoundError: No module named 'implicit'

In [3]:
pip install implicit

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [6]:
data = pd.read_csv('C:/Users/Home/OneDrive/Рабочий стол/DF.Rec.sys/data/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


Задание 1. Weighted Random Recommendation

In [11]:
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()


In [12]:
items_weights.head()

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98


In [13]:
import random

In [14]:
def weighted_random_recommendation(items_weights, n=5):
    #Взвешенные cлучайные рекоммендации
    items_weights = items_weights.sort_values(by='sales_value', ascending=False)
    items_weights['weight']=np.log10(items_weights['sales_value']+1)/100
    recs = random.choices(items_weights['item_id'], weights=items_weights['weight'], k=n)
    return recs

In [15]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
result['weighted_random_recommendation'] = result['user_id'].map(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head()

Wall time: 55 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1122545, 110450, 100186, 1009567, 983800]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[723354, 13039382, 66398, 884372, 1765756]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[567012, 928241, 891798, 2212751, 871162]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1057257, 1325256, 976593, 896369, 910689]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1054377, 9529723, 1013848, 1004760, 112609]"


Задание 2. Расчет метрик

In [16]:
def random_recommendation(items, n=5):
    """Случайные рекомендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [17]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].map(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 4.54 s


,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1122545, 110450, 100186, 1009567, 983800]","[1130991, 15972567, 9301492, 12582356, 9704731]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[723354, 13039382, 66398, 884372, 1765756]","[1130756, 1121868, 1114894, 1053340, 6545205]"


In [18]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [19]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].map(lambda x: popular_recs)
result.head(2)


Wall time: 158 ms


,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1122545, 110450, 100186, 1009567, 983800]","[1130991, 15972567, 9301492, 12582356, 9704731]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[723354, 13039382, 66398, 884372, 1765756]","[1130756, 1121868, 1114894, 1053340, 6545205]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [20]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [21]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [22]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id']

6           1043142
8           1102651
9           6423775
10          9487839
12           897044
             ...   
2282311    13877226
2282314     1057256
2282315    12487271
2282316      836262
2282321      993339
Name: item_id, Length: 829774, dtype: int64

In [24]:
# Заведем фиктивный item_id (если юзер покупал товары не из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [26]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 472 ms


In [28]:
recs

(array([3587, 2307, 2148, 3947, 3408]),
 array([ 7431., 12981., 43806., 17219., 56269.]))

In [29]:
[id_to_itemid[rec] for rec in recs[0]]

[1098066, 995242, 981760, 1127831, 1082185]

In [30]:
%%time
result['itemitem'] = result['user_id'].\
    map(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)[0]])

Wall time: 96 ms


In [32]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1122545, 110450, 100186, 1009567, 983800]","[1130991, 15972567, 9301492, 12582356, 9704731]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[723354, 13039382, 66398, 884372, 1765756]","[1130756, 1121868, 1114894, 1053340, 6545205]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]"


In [33]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

D:\Anaconda\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0059964656829833984 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 741 ms


In [34]:
[id_to_itemid[rec] for rec in recs[0]]

[961554, 1098066, 1127831, 981760, 1082185]

In [35]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

Wall time: 98.5 ms


In [36]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1122545, 110450, 100186, 1009567, 983800]","[1130991, 15972567, 9301492, 12582356, 9704731]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[723354, 13039382, 66398, 884372, 1765756]","[1130756, 1121868, 1114894, 1053340, 6545205]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]"


In [38]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

D:\Anaconda\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.005000114440917969 seconds
  warnings.warn(


  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 757 ms


In [39]:
[id_to_itemid[rec] for rec in recs[0]]

[961554, 1098066, 1127831, 981760, 1082185]

In [40]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

Wall time: 100 ms


In [41]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1122545, 110450, 100186, 1009567, 983800]","[1130991, 15972567, 9301492, 12582356, 9704731]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[723354, 13039382, 66398, 884372, 1765756]","[1130756, 1121868, 1114894, 1053340, 6545205]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]"


In [42]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)[0]])

Wall time: 115 ms


In [43]:
result.head(2)


,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1122545, 110450, 100186, 1009567, 983800]","[1130991, 15972567, 9301492, 12582356, 9704731]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[723354, 13039382, 66398, 884372, 1765756]","[1130756, 1121868, 1114894, 1053340, 6545205]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 995242, 981760, 1127831, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]","[961554, 1098066, 1127831, 981760, 1082185]"


In [44]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [45]:
rr_at_5 = round(result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean(), 5)
rr_at_5

0.00108

In [46]:
0.00088
pr_at_5 = round(result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean(), 5)
pr_at_5

0.15524

In [47]:
wrr_at_5 = round(result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean(), 5)
wrr_at_5

0.00118

In [48]:
ii_at_5 = round(result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean(), 5)
ii_at_5

0.14574

In [49]:
cos_at_5 = round(result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean(), 5)
cos_at_5

0.13546

In [50]:
tfidf_at_5 = round(result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean(), 5)
tfidf_at_5

0.13546

In [51]:
op_at_5 = round(result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean(), 5)
op_at_5

0.13546

In [52]:
prec_at_5 = [rr_at_5, pr_at_5, wrr_at_5, ii_at_5, cos_at_5, tfidf_at_5, op_at_5]

In [53]:
rr_at_3 = round(result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual'], 3), axis=1).mean(), 5)
rr_at_3

0.00098

In [54]:
pr_at_3 = round(result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual'], 3), axis=1).mean(), 5)
pr_at_3

0.13777

In [55]:
wrr_at_3 = round(result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual'], 3), axis=1).mean(), 5)
wrr_at_3

0.00131

In [57]:
ii_at_3 = round(result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'], 3), axis=1).mean(), 5)
ii_at_3

0.11965

In [58]:
cos_at_3 = round(result.apply(lambda row: precision_at_k(row['cosine'], row['actual'], 3), axis=1).mean(), 5)
cos_at_3

0.05893

In [59]:
tfidf_at_3 = round(result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'], 3), axis=1).mean(), 5)
tfidf_at_3

0.05893

In [60]:
op_at_3= round(result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual'], 3), axis=1).mean(), 5)
op_at_3

0.05893

In [61]:
prec_at_3 = [rr_at_3, pr_at_3, wrr_at_3, ii_at_3, cos_at_3, tfidf_at_3, op_at_3]

In [62]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list,bought_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [63]:
rr_rec_at_5 = round(result.apply(lambda row: recall_at_k(row['random_recommendation'], row['actual']), axis=1).mean(), 5)
rr_rec_at_5


0.00014

In [64]:
pr_rec_at_5 = round(result.apply(lambda row: recall_at_k(row['popular_recommendation'], row['actual']), axis=1).mean(), 5)
pr_rec_at_5


0.025

In [65]:
wrr_rec_at_5 = round(result.apply(lambda row: recall_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean(), 5)
wrr_rec_at_5

0.0003

In [66]:
ii_rec_at_5 = round(result.apply(lambda row: recall_at_k(row['itemitem'], row['actual']), axis=1).mean(), 5)
ii_rec_at_5

0.01622

In [67]:
cos_rec_at_5 = round(result.apply(lambda row: recall_at_k(row['cosine'], row['actual']), axis=1).mean(), 5)
cos_rec_at_5


0.01403

In [68]:
tfidf_rec_at_5 = round(result.apply(lambda row: recall_at_k(row['tfidf'], row['actual']), axis=1).mean(), 5)
tfidf_rec_at_5

0.01403

In [69]:
op_rec_at_5 = round(result.apply(lambda row: recall_at_k(row['own_purchases'], row['actual']), axis=1).mean(), 5)
op_rec_at_5

0.01403

In [70]:
rec_at_5 = [rr_rec_at_5, pr_rec_at_5, wrr_rec_at_5, ii_rec_at_5, cos_rec_at_5, tfidf_rec_at_5, op_rec_at_5]

Результаты

In [72]:
metrics = pd.DataFrame([prec_at_5, prec_at_3, rec_at_5], index = ['precision@5', 'precision@3', 'recall@5'], columns = ['random_recommendation', 'popular_recommendation', 'weighted_random_recommendation', 'itemitem', 'cosine', 'tfidf', 'own_purchases'])
metrics

,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases
precision@5,0.00108,0.15524,0.00118,0.14574,0.13546,0.13546,0.13546
precision@3,0.00098,0.13777,0.00131,0.11965,0.05893,0.05893,0.05893
recall@5,0.00014,0.02500,0.00030,0.01622,0.01403,0.01403,0.01403
